## Importing libraries

In [2]:
import numpy as np
import pandas as pd
import scipy.linalg as la
import sympy as sp
import itertools
import math

In [101]:
data=pd.read_excel("/home/sanvi/Downloads/Book2.xlsx")
matrix = data.iloc[:,:].values
matrix = matrix.reshape (5,5)

In [102]:
matrix = np.array (matrix, dtype= float)
print("The matrix is: ")
matrix

The matrix is: 


array([[ -218.4723,  -901.2143,   845.2553,  -282.1788,   386.958 ],
       [  -50.8714,  -513.7404,   773.4364,  -508.1207,   255.1108],
       [ -257.6493, -1164.8854,  1406.7087,  -647.3382,   461.3214],
       [ -479.7835, -1067.586 ,   992.9244,  -100.9406,   449.2103],
       [ -400.3896,  -736.7248,   677.3835,   -89.1507,   450.5357]])

In [103]:
if (np.any(np.iscomplex(la.eig(matrix)[0]))):
    raise ValueError ("Matrix has complex eigenvalues.")

## (a) Determine the eigenvalues of A by the LU method.

### LU decomposition 

In [104]:
def lud(a):
    n = a.shape[0]
    l = np.zeros((n, n))
    u = np.zeros((n, n))
    np.fill_diagonal(l, 1)
    u[0] = a[0]

    for i in range(1, n):
        for j in range(n):
            if i <= j:
                u[i][j] = a[i][j] - sum(u[k][j] * l[i][k] for k in range(i))
            if i > j:
                l[i][j] = (a[i][j] - sum(u[k][j] * l[i][k] for k in range(j))) / u[j][j]
                
    return l, u

In [105]:
matrix

array([[ -218.4723,  -901.2143,   845.2553,  -282.1788,   386.958 ],
       [  -50.8714,  -513.7404,   773.4364,  -508.1207,   255.1108],
       [ -257.6493, -1164.8854,  1406.7087,  -647.3382,   461.3214],
       [ -479.7835, -1067.586 ,   992.9244,  -100.9406,   449.2103],
       [ -400.3896,  -736.7248,   677.3835,   -89.1507,   450.5357]])

In [106]:
l, u= lud (matrix)

### L matrix

In [107]:
l

array([[ 1.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.23285057,  1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.1793225 ,  0.33585303,  1.        ,  0.        ,  0.        ],
       [ 2.1960839 , -2.99960449,  4.00654398,  1.        ,  0.        ],
       [ 1.83267902, -3.01064632,  3.99727532,  1.67791568,  1.        ]])

### U matrix

In [108]:
u

array([[-218.4723    , -901.2143    ,  845.2553    , -282.1788    ,
         386.958     ],
       [   0.        , -303.89213484,  576.61821999, -442.41520504,
         165.0074084 ],
       [   0.        ,    0.        ,  216.22112565, -165.97190322,
         -50.44511597],
       [   0.        ,    0.        ,    0.        , -143.34918771,
         296.48560591],
       [   0.        ,    0.        ,    0.        ,    0.        ,
         -57.68999094]])

### Shifting

In [109]:
def shift(A):
    possible_shift_vals = []
    
    for i in range(np.shape(A)[0]):
        up_lim = A[i][i]
        low_lim = A[i][i] 
        
        for j in range(np.shape(A)[0]):
            if i != j :
                up_lim=up_lim+abs(A[i][j])
                low_lim=low_lim-abs(A[i][j])
                
        possible_shift_vals.append(up_lim )
        possible_shift_vals.append(low_lim)    

    shift=np.max(np.abs(possible_shift_vals))
    return shift

### Finding eigen values

In [131]:
def UL_eigen (A, iters= 50000, tol = 1e-15):
    m,n = A.shape 
    I = np.identity (np.shape(A)[0])
    shift_A = shift(A) + 1
    A = A + I * (shift_A)
    
    D1 = A ; D2 = np.ones(np.shape(A))
    iter = 0
  
    while (np.allclose(np.diagonal (D1), np.diagonal (D2), tol)==False) :
        L,U = lud(D1)
        D2 = np.matmul (U,L)
        
        if (np.allclose(np.diagonal (D1), np.diagonal (D2), tol)==True):
            return np.diagonal(D2) -(shift_A)
            
        D1 = D2
        D2 = np.zeros((m,n))
        iter = iter + 1

        if (iter > iters):
            raise ValueError ("System fails to converge after 50000 iterations. Try another matrix")
            return "NA"

In [111]:
UL_eigen (matrix)
eigen = np.sort(UL_eigen (matrix))
print(f"The Eigen values of the matrix are {eigen}")

The Eigen values of the matrix are [ 55.89979633 115.91541822 148.5156468  330.51883915 373.2413995 ]


## (b) Using these eigenvalues, determine the determinant and indicate the uniqueness of the system.

In [112]:
det=np.prod(eigen)
det=np.round(det,3)
print(f"The determinant of a matrix [|A|] is {det:0.4f}")

if det==0 :
    print("The system is inconsistent and  has either no solutions or infinite solutions.")
else:
    print("The system is consistent and has unique solutions.")

The determinant of a matrix [|A|] is 118716113663.9150
The system is consistent and has unique solutions.


## (c) Using these eigenvalues, determine the condition number of this matrix and compare it with the condition number of the Hilbert matrix.

In [113]:
def condition_num(eigenvalues):
    max = np.max(eigenvalues)
    min = np.min(eigenvalues)
    condition_number = max/min
    n=len(eigenvalues)
    hilbert = np.zeros([n,n])
    for i in range(0,n):
        for j in range(0,n):
            hilbert[i][j] = 1/(i+j+1)
    
    eigen_hilbert = UL_eigen(hilbert)
    condition_hilbert = np.max(eigen_hilbert)/np.min(eigen_hilbert)
    print(f"Condition number of hilbert matrix = {condition_hilbert}")
    if condition_hilbert > condition_number:
        print(f'It is well-conditioned matrix with condition number = {condition_number}.')
    else:
        print(f'It is ill - conditioned matrix with condition number = {condition_number}. ')
    return hilbert

In [114]:
hilbert = condition_num (eigen)

Condition number of hilbert matrix = 13304.948112154536
It is well-conditioned matrix with condition number = 6.676972439727691.


In [115]:
print (hilbert)

[[1.         0.5        0.33333333 0.25       0.2       ]
 [0.5        0.33333333 0.25       0.2        0.16666667]
 [0.33333333 0.25       0.2        0.16666667 0.14285714]
 [0.25       0.2        0.16666667 0.14285714 0.125     ]
 [0.2        0.16666667 0.14285714 0.125      0.11111111]]


In [116]:
print('This formula of Condition Number = (max eigenvalue) / (min eigenvalue) is only applicable if A is a normal matrix. Otherwise, we use function of (maximum singular value) / (minimum singular value), where singular values can be found by performing SVD of the matrix.')

_,s,_ = la.svd (matrix)
_,s1,_ = la.svd (hilbert)
print (f"\nIf calculated using SVD, we get: Matrix condition number = {s[0]/s[-1]:0.4f} and Hilbert condition number = {s1[0]/s1[-1]:0.4f}.")

This formula of Condition Number = (max eigenvalue) / (min eigenvalue) is only applicable if A is a normal matrix. Otherwise, we use function of (maximum singular value) / (minimum singular value), where singular values can be found by performing SVD of the matrix.

If calculated using SVD, we get: Matrix condition number = 735.2655 and Hilbert condition number = 476607.2502.


## (d) Write down the polynomial equation for which these eigenvalues are the roots of the polynomial.

In [117]:
product_of_roots=[1]   #product of roots taken n at a time
dimension=np.shape(matrix)[0]

for i in range(dimension):
    # Generate all pairs of elements
    combination = itertools.combinations(eigen, i+1)
    
    # Calculate the product of each pair
    products = [math.prod(roots) for roots in combination]
    product_of_roots.append(((-1)**(i+1))*np.sum(products))
    
n = len(product_of_roots)
x = sp.symbols("x")

for i in range(1,n+1):
    print(f"({(np.sum(product_of_roots[i-1])):0.3f} x^{n-i})", end = " + ")
print(0)

(1.000 x^5) + (-1024.091 x^4) + (380796.334 x^3) + (-62997548.971 x^2) + (4624492124.675 x^1) + (-118716113663.915 x^0) + 0


## (e) Determine the eigenvalue by the power method for both A and the inverse of A. Determine the inverse of A by the Jordan technique. Show that these eigenvalues match with the ones obtained by the LU method.

In [118]:
zero_check = 0

In [119]:
def gauss_jordan(sample_matrix):
    global zero_check
    n = len(sample_matrix)
    augmented_matrix = np.hstack((sample_matrix, np.identity(n)))  # Augment with identity matrix

    for i in range(n):
        # Partial pivoting: Find the maximum element in the current column
        max_row = np.argmax(np.abs(augmented_matrix[i:, i])) + i
        if augmented_matrix[max_row, i] == 0:
            raise ValueError("Matrix is singular and cannot be inverted.")
            zero_check = 1
                        
        # Swap rows to place the largest element on the diagonal
        if i != max_row:
            augmented_matrix[[i, max_row]] = augmented_matrix[[max_row, i]]
        
        # Make the diagonal element 1 by dividing the row by the diagonal element
        diag_element = augmented_matrix[i][i]
        augmented_matrix[i] = augmented_matrix[i] / diag_element

        # Make all elements in the current column, except the diagonal element, 0
        for j in range(n):
            if i != j:
                multiplication_factor = augmented_matrix[j][i]
                augmented_matrix[j] = augmented_matrix[j] - multiplication_factor * augmented_matrix[i]

    # Extract the inverse matrix from the augmented matrix
    inverse_matrix = augmented_matrix[:, n:]
    return inverse_matrix

In [120]:
def power_method (A, x_input, iters=5000, tol = 1e-15):
    b = np.zeros ((np.array(x_input).shape[0], iters))
    c = np.zeros ((iters)); iter = 0
    
    b_in = np.matmul (A, x_input) 
    c[0] = np.max (b_in)
    x_in = b_in / c[0]

    for i in range (iters):
        iter = iter + 1
        b = np.matmul(A, x_in)
        c[i+1] = np.max (b)
        x = b / c[i+1] 
        
        if (iter > iters - 2):
            raise ValueError (f"System not converged after {iters} iterations.")
            return 'NA'
            
        if (np.allclose(c[i+1], c[i], tol)):
            return c[i+1]
            
        elif (np.allclose(c[i+1], c[i-1], tol)):
            c_max = np.sqrt(c[i+1]*c[i-1])
            return c_max            
        else:
            c[i+2] = c [i+1]
            x_in = x

In [121]:
def initial_vector(A):
    matrix = []; n = A.shape [0]
    for i in range(n):
        while True:
            try:
                value = float (input (f"Enter value for [{i}]: "))
                matrix.append(value)
                break
                
            except ValueError:
                print("Error: Please enter a valid float number.")
    return matrix

In [122]:
det = np.round(UL_eigen(matrix),4)
if np.allclose(det,0):
    raise ValueError ("Determinant = 0, Matrix is singular, non-invertible.")
    zero_check = 1
else:
    matrix_inv = gauss_jordan(matrix)

if (zero_check == 0):
    print ("Inverse of matrix calculated using Gauss-Jordan: ")
    print (matrix_inv)

Inverse of matrix calculated using Gauss-Jordan: 
[[ 0.01904828  0.07451485 -0.0694239   0.04861467 -0.03593923]
 [ 0.00269396  0.04400406 -0.03783424  0.02870269 -0.01710888]
 [ 0.00817948  0.06790603 -0.059944    0.04760667 -0.03156383]
 [ 0.01371895  0.07501115 -0.06975911  0.0531799  -0.03585155]
 [ 0.01175011  0.05092322 -0.0472416   0.02908504 -0.01733403]]


In [123]:
print("Initial vector for power method")
a = initial_vector (matrix)

Initial vector for power method


Enter value for [0]:  1
Enter value for [1]:  0
Enter value for [2]:  0
Enter value for [3]:  1
Enter value for [4]:  0


#### Verifying if the eigen values found using power method and LU are the same 

##### For A

In [124]:
print("Finding eigen value by power method for matrix(A)")
pow = power_method (matrix,a)

if (isinstance (pow, float)):
    print(f"\nThe largest eigen value is: {pow}")
    
    print( f"The largest eigen value by LU decomposition is {np.max(UL_eigen(matrix))}")
    
    if np.allclose(pow,np.max(UL_eigen(matrix)), atol=1e-3 )== True:
        print("\nEigen values are matching.")

Finding eigen value by power method for matrix(A)

The largest eigen value is: 373.2413994206315
The largest eigen value by LU decomposition is 373.2413994974372

Eigen values are matching.


##### For A inverse

In [125]:
la.eig (matrix_inv)

(array([0.01788915+0.j, 0.00862698+0.j, 0.0067333 +0.j, 0.00302555+0.j,
        0.00267923+0.j]),
 array([[ 0.56959627,  0.59233232,  0.03419653, -0.11698838,  0.43595076],
        [ 0.22872752,  0.13333189,  0.58171331, -0.62663859,  0.09049141],
        [ 0.43176391,  0.28148342,  0.34785624, -0.75536034,  0.38179245],
        [ 0.53712306,  0.48609614,  0.13075306, -0.1333673 ,  0.63313465],
        [ 0.38512666,  0.56199881,  0.72273086, -0.07267541,  0.5051072 ]]))

In [126]:
print("Finding eigen value by power method for matrix(A inverse)")
pow_inv = power_method (matrix_inv,a)

if (isinstance (pow, float)):
    print(f"\nThe largest eigen value is: {pow_inv}")
    print(f"The largest eigen value by LU decomposition is {np.max(UL_eigen(matrix_inv))}")
    
    if np.allclose(1/pow_inv,np.min(UL_eigen(matrix)) ,atol=1e-3)== True:
        print("\nEigen values are matching.")

Finding eigen value by power method for matrix(A inverse)

The largest eigen value is: 0.01788915841853292
The largest eigen value by LU decomposition is 0.017889152722392554

Eigen values are matching.


## (f) Solve Ax=b for any b. Change to another b and use the same LU decomposed matrix to solve the system.

In [127]:
def solve_LU(sample_matrix, b):
    L,U = lud(sample_matrix)
    y = np.zeros(len(b))
    x = np.zeros(len(b))
    
    # Ly = b
    y[0] = b[0]
    for i in range(1, len(b)):
        sum_Ly = 0
        for j in range(i):
            sum_Ly += L[i][j] * y[j]
        y[i] = b[i] - sum_Ly

    #Ux = y
    x[-1] = y[-1] / U[-1][-1]
    for i in range(len(b) - 2, -1, -1):
        sum_Ux = 0
        for j in range(i + 1, len(b)):
            sum_Ux += U[i][j] * x[j]
        x[i] = (y[i] - sum_Ux) / U[i][i]

    print("Solution vector x:", x)
    return x

In [128]:
def b(A):
    matrix = []; n = A.shape [0]
    for i in range(n):
        while True:
            try:
                value = float (input (f"Enter value for [{i}]: "))
                matrix.append(value)
                break
                
            except ValueError:
                print("Error: Please enter a valid float number.")
    return matrix

In [129]:
b = b (matrix)

Enter value for [0]:  1
Enter value for [1]:  5
Enter value for [2]:  3
Enter value for [3]:  8
Enter value for [4]:  3


In [130]:
x = solve_LU (matrix, b)

Solution vector x: [0.4644505  0.28750647 0.45403948 0.49738194 0.30531963]
